In [1]:
import driver
from pynq.pl_server.device import Device

import os
import my_metrics
import pandas as pd

# Images List and Labels Folder

### DFire

In [2]:
dfire_imgs_folder = '../../../../datasets/dfire_test/images/'
dfire_imgs_list = [dfire_imgs_folder + fname for fname in os.listdir(dfire_imgs_folder) if not fname.startswith('.')] 

dfire_labels_folder = '../../../../datasets/dfire_test/labels/'

In [3]:
print(len(dfire_imgs_list))

4306


### FASDD UAV

In [4]:
fasdd_uav_imgs_folder = '../../../../datasets/fasdd_uav_test/'
fasdd_uav_imgs_list = [fasdd_uav_imgs_folder + fname for fname in os.listdir(fasdd_uav_imgs_folder) if not fname.startswith('.')] 

In [5]:
print(len(fasdd_uav_imgs_list))

4181


### FASDD CV

In [6]:
fasdd_cv_imgs_folder = '../../../../datasets/fasdd_cv_test/'
fasdd_cv_imgs_list = [fasdd_cv_imgs_folder + fname for fname in os.listdir(fasdd_cv_imgs_folder) if not fname.startswith('.')] 

In [7]:
print(len(fasdd_cv_imgs_list))

15884


# Dataset Dictionary

In [8]:
datasets_dict = {
    "dfire": dfire_imgs_list,
    "fasdd_uav": fasdd_uav_imgs_list,
    "fasdd_cv": fasdd_cv_imgs_list,
}

# Accelerator

In [9]:
my_device = Device.devices[0]

accel = driver.FINNExampleOverlay(
    bitfile_name = '../bitfile/finn-accel.bit', 
    platform = "zynq-iodma",
    io_shape_dict = driver.io_shape_dict, 
    batch_size = 1,
    runtime_weight_dir = "runtime_weights/", 
    device=my_device
)

# Throughput Test

In [10]:
res = accel.throughput_test()
file = open("nw_metrics.txt", "w")
file.write(str(res))
file.close()
print("Results written to nw_metrics.txt")

Results written to nw_metrics.txt


In [11]:
for key, value in res.items():
    print(f'{key}: {value}')

runtime[ms]: 2.3360252380371094
throughput[images/s]: 428.07756685037765
DRAM_in_bandwidth[MB/s]: 64.43765998285365
DRAM_out_bandwidth[MB/s]: 0.0008561551337007553
fclk[mhz]: 100.0
batch_size: 1
fold_input[ms]: 0.12540817260742188
pack_input[ms]: 0.09608268737792969
copy_input_data_to_device[ms]: 2.1517276763916016
copy_output_data_from_device[ms]: 0.2760887145996094
unpack_output[ms]: 2.8519630432128906
unfold_output[ms]: 0.04935264587402344


# Metrics

In [13]:
# import importlib
# importlib.reload(my_metrics)

<module 'my_metrics' from '/home/xilinx/jupyter_notebooks/uav/code/mobilenetv2_resnet/750_FPS/driver/my_metrics.py'>

### FULL Test Dataset

In [14]:
metrics_res = my_metrics.eval_accel(
    datasets_dict = datasets_dict,
    dfire_labels_dir = dfire_labels_folder,
    accel = accel
)

Number of test samples dfire: 4306



Testing: 100%|██████████| 4306/4306 [07:13<00:00,  9.94it/s]


Number of test samples fasdd_uav: 4181



Testing: 100%|██████████| 4181/4181 [32:02<00:00,  2.18it/s]


Number of test samples fasdd_cv: 15884



Testing: 100%|██████████| 15884/15884 [29:05<00:00,  9.10it/s] 


Smoke -> TP: 10763.0, FP: 522.0, TN: 12359.0, FN: 727.0
Fire  -> TP: 7759.0, FP: 482.0, TN: 15987.0, FN: 143.0


In [15]:
metrics_df = pd.DataFrame(metrics_res)
metrics_df

,Smoke,Fire,Mean
Accuracy,0.9488,0.9744,0.9616
Precision,0.9537,0.9415,0.9476
Recall,0.9367,0.9819,0.9593
F1,0.9452,0.9613,0.9532
